In [1]:
import pandas as pd 

from sklearn import metrics

import seaborn as sns

# sklearn :: utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# sklearn :: models
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

sns.set_style('whitegrid')

In [2]:
tracks = pd.read_csv('../../data/processed/tracks_processed.csv', dtype=object)
tracks.head()

,Unnamed: 0,track_id,album_date_created,album_date_released,album_favorites,album_id,album_listens,album_tags,album_title,album_tracks,...,track_language_code,track_listens,track_number,track_title,track_year_created,bit_rate_factor,listens_factor,interest_factor,track_price,track_length
0,0,2,2008-11-26 1:44,2009-01-05 0:00,4,1,6073,[],AWOL - A Way Of Life,7,...,en,1293,3,Food,2008,0.7,2,1.0,3.49,less than 3 minutes
1,1,3,2008-11-26 1:44,2009-01-05 0:00,4,1,6073,[],AWOL - A Way Of Life,7,...,en,514,4,Electric Ave,2008,0.7,3,0.1,0.35,between 3 and 5 minutes
2,2,5,2008-11-26 1:44,2009-01-05 0:00,4,1,6073,[],AWOL - A Way Of Life,7,...,en,1151,6,This World,2008,0.7,2,0.7,2.45,between 3 and 5 minutes
3,3,10,2008-11-26 1:45,2008-02-06 0:00,4,6,47632,[],Constant Hitmaker,2,...,en,50135,1,Freeway,2008,0.5,1,1.0,2.5,less than 3 minutes
4,4,20,2008-11-26 1:45,2009-01-06 0:00,2,4,2710,[],Niris,13,...,en,361,3,Spiritual Level,2008,0.7,4,0.1,0.35,between 5 and 7 minutes


# Feature Engineering

In [3]:
# modeling based on Track_listens, track_year_created, and interest_factor
X_columns=[tracks.columns.get_loc(c) for c in tracks.columns if c in ["track_listens", "track_year_created"]]
y_column = tracks.columns.get_loc('interest_factor')

# Model Training

In [5]:
# split the data

threshold = 0.8
absolute_threshold = int(len(tracks)*threshold)

X_train = tracks.ix[:absolute_threshold, X_columns]
y_train = tracks.ix[:absolute_threshold, y_column]

X_test = tracks.ix[absolute_threshold:, X_columns]
y_test = tracks.ix[absolute_threshold:, y_column]

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (84720, 2)
y_train (84720,)
X_test (21180, 2)
y_test (21180,)


In [6]:
models = [
    ('Naive Bayes', GaussianNB()),
    ('RandomForestClassifier10', RandomForestClassifier(n_estimators=10)),
    ('RandomForestClassifier100', RandomForestClassifier(n_estimators=100)),
    ('KNeighborsClassifier', KNeighborsClassifier()),
    ('DecisionTreeClassifier', DecisionTreeClassifier())
]
results = []
for m in models:
    print('MODEL', m[0])
    model = m[1]
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred, average = 'micro')
    recall = recall_score(y_test, y_pred, average = 'micro')
    print(confusion_matrix(y_test, y_pred))
    print('Precision', precision)
    print('Recall', recall)
    results.append([m[0], precision, recall])
    
    # if there is a feature importance, print top 5
    importance = []
    if hasattr(model, 'feature_importances_'):
        print('Feature Importance')
        importance = []
        for i in range(len(X_columns)):
            importance.append([X_columns[i], model.feature_importances_[i]])
        print(pd.DataFrame(importance).sort_values(by=1, ascending=False).head())
    elif hasattr(model, 'coef_'):
        print('Feature Importance')
        for i in range(len(X_columns)):
            importance.append([X_columns[i], model.coef_[i]])
        print(pd.DataFrame(importance).sort_values(by=1, ascending=False).head())
        
    print('')

# sort the results and print as a table
df_results = pd.DataFrame(results)
df_results.columns = ['model', 'precision', 'recall']
df_results = df_results.sort_values(by='precision', ascending=False)
df_results

MODEL Naive Bayes
[[1827 1961   22 1667    0    0]
 [   0 2258 1088    0    0    0]
 [1798   45  751    0    0    0]
 [  15    2    4 2946   81    0]
 [   1    1    0  196  453   33]
 [  39   11   14   97  255 5615]]
Precision 0.653918791312559
Recall 0.653918791312559

MODEL RandomForestClassifier10
[[3708 1104  151  502    0   12]
 [  23 2796  524    0    0    3]
 [ 554  127 1906    0    0    7]
 [ 136    4    5 2558  238  107]
 [   1    1    0  106  275  301]
 [  44   16   20   68  124 5759]]
Precision 0.802738432483475
Recall 0.802738432483475
Feature Importance
    0         1
0  41  0.977522
1  44  0.022478

MODEL RandomForestClassifier100
[[3755 1087  120  508    0    7]
 [  25 2805  516    0    0    0]
 [ 581  125 1888    0    0    0]
 [ 115    4    5 2568  245  111]
 [   1    1    0   95  290  297]
 [  46   16   18   67  117 5767]]
Precision 0.8060906515580737
Recall 0.8060906515580737
Feature Importance
    0         1
0  41  0.972333
1  44  0.027667

MODEL KNeighborsClassifi

,model,precision,recall
3,KNeighborsClassifier,0.822380,0.822380
4,DecisionTreeClassifier,0.812040,0.812040
2,RandomForestClassifier100,0.806091,0.806091
1,RandomForestClassifier10,0.802738,0.802738
0,Naive Bayes,0.653919,0.653919
